In [1]:
import pandas as pd

# 기존 데이터 파일 읽기
input_file = "CrowledData.csv"
data = pd.read_csv(input_file)

# 필요한 열만 추출 (인덱스 번호, 카페이름, URL)
output_data = data[['카페이름', 'url']].drop_duplicates().reset_index(drop=True)
output_data.reset_index(inplace=True)
output_data.rename(columns={'index': '인덱스번호'}, inplace=True)

# 새로운 CSV 파일 저장
output_file = "Cafe_Data.csv"
output_data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"새로운 파일이 생성되었습니다: {output_file}")


새로운 파일이 생성되었습니다: Cafe_Data.csv


In [ ]:
import pandas as pd

df=pd.read_csv("CrowledData.csv", encoding='utf-8-sig')

df_clean = df[df['sentiment'].isin(['Positive', 'Very Positive', 'Neutral'])]

# 카페별 리뷰 통합
cafe_reviews = df_clean.groupby('카페이름')['review'].apply(lambda x: ' '.join(x)).reset_index()

df_clean

,Column1,카페이름,url,홈url,review,sentiment,confidence
1,2,투또톤토,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,아침먹고 투또톤토에 갔어요~ 여유롭게 시간 보내기가 좋아요~ 포장하면 할인되고요~ㅎ...,Neutral,0.4055
2,4,투또톤토,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,매장이 엄청 넓고 인테리어도 모던하니 예뻤어요ㅎㅎ\n커피에 진짜 진심인 카페였습니다...,Very Positive,0.7247
3,6,투또톤토,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,건대입구 오픈했을때부터 애용하는 카페 투또톤토입니다!(아직도 이름이 헷갈려요)\n\...,Very Positive,0.4722
4,8,투또톤토,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,아침 일찍오니 빵 종류가 많네요. 좋아하는 말차빵이 있어 얼른 get했어요. 샐러드...,Positive,0.3388
5,9,투또톤토,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,쌀쌀한 날 따뜻한 라떼가 너무 맛있어요.\n카페에 들어왔을 때 나는 빵 냄새랑 커피...,Very Positive,0.5238
...,...,...,...,...,...,...,...
16039,20878,카페롬곡카페,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,와플 맛집!!! 세종대 근처에서 와플이 먹고싶을때 자주 옵니다아,Very Positive,0.5783
16040,20879,카페롬곡카페,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,재방문인데 커피랑 빙수가 진짜 맛있어요 빙수전문카페보다 맛있습니다 사장님 더 대박나세용,Neutral,0.3012
16041,20880,카페롬곡카페,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,와플 전문집,Neutral,0.3901
16042,20882,카페롬곡카페,https://map.naver.com/p/search/건대입구 카페/place/1...,https://map.naver.com/p/search/%EA%B1%B4%EB%8C...,분위기가 따뜻하고 좋아요\n사장님이 친절하세요,Positive,0.4618


In [3]:
import pandas as pd
from transformers import pipeline

# 데이터 로드
df = pd.read_csv("CrowledData.csv")

# 감성 분석 모델 로드
classifier = pipeline(
    "sentiment-analysis", model="sangrimlee/bert-base-multilingual-cased-nsmc")

# tqdm으로 진행 상황 표시
from tqdm import tqdm

# 빈 리스트 생성
sentiment_results = []

# 처음 50개의 리뷰만 처리하도록 수정
# df의 전체 길이가 50보다 작으면 전체를, 아니면 처음 50개만
max_reviews = min(300, len(df))
for review in tqdm(df['review'].head(max_reviews).fillna('')):
    try:
        # 빈 문자열이거나 너무 짧은 리뷰는 'Unknown'으로 처리
        if len(review) < 1:
            sentiment_results.append('Unknown')
        else:
            # 길이가 긴 리뷰는 처음 512 토큰만 사용
            result = classifier(review[:512])[0]
            sentiment_results.append(result['label'])
    except Exception as e:
        print(f"Error processing review: {e}")
        sentiment_results.append('Error')

# 앞의 50개 결과만 저장하고 나머지는 NaN으로 채우기
df_sample = df.copy()
df_sample['senti_2'] = pd.NA  # 모든 행을 NaN으로 초기화
df_sample.loc[:max_reviews-1, 'senti_2'] = sentiment_results  # 앞의 50개만 결과 저장

# 결과 저장 (테스트용이므로 다른 파일명으로 저장)
df_sample.to_csv("CrowledData_with_sentiment_sample.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df_sample[['review', 'senti_2']].head(max_reviews))
print(f"분석 완료! 앞의 {max_reviews}개 리뷰만 처리했습니다.")

Device set to use cpu
100%|██████████| 300/300 [00:30<00:00,  9.82it/s]


                                                review   senti_2
0    자리는 불편한데 빵은 맛있는 카페\n\n롤토르타(레몬)\n레몬향이 한가득 나서 초반...  negative
1    아침먹고 투또톤토에 갔어요~ 여유롭게 시간 보내기가 좋아요~ 포장하면 할인되고요~ㅎ...  positive
2    매장이 엄청 넓고 인테리어도 모던하니 예뻤어요ㅎㅎ\n커피에 진짜 진심인 카페였습니다...  positive
3    건대입구 오픈했을때부터 애용하는 카페 투또톤토입니다!(아직도 이름이 헷갈려요)\n\...  positive
4    아침 일찍오니 빵 종류가 많네요. 좋아하는 말차빵이 있어 얼른 get했어요. 샐러드...  positive
..                                                 ...       ...
295  성수역은 팝업스토어로 가끔 갔었는데 성수동 카페 까치화방을 방문하기는 처음이었어요\...  positive
296  성수 카페 까치화방 인테리어 정말 멋지고 다양한 케이크는 호텔급 퀄리티예요! 음료와...  positive
297  성심당 딸기시루 스타일 케이크가 있다고 해서 예약후 방문했는데 카페 인테리어도 독특...  positive
298  까치화방 성수카페맛집 성수낙낙 제 최애 카페 중 하나에요.. 인테리어도 너무 예쁘고...  positive
299  케이크 종류도 크고 다양하고 ㅎㅎ\n기념일에 케이크 사기 좋을듯요 !\n그외에 베이...  positive

[300 rows x 2 columns]
분석 완료! 앞의 300개 리뷰만 처리했습니다.


In [4]:
import pandas as pd
from transformers import pipeline

# 데이터 로드
df = pd.read_csv("CrowledData.csv")

# 감성 분석 모델 로드
classifier = pipeline(
    "sentiment-analysis", model="sangrimlee/bert-base-multilingual-cased-nsmc")

# 리뷰 컬럼이 5번째 열이라면 (인덱스 4)
# tqdm으로 진행 상황 표시
from tqdm import tqdm

# 빈 리스트 생성
sentiment_results = []

# 각 리뷰에 대해 감성 분석 수행
for review in tqdm(df['review'].fillna('')):
    try:
        # 빈 문자열이거나 너무 짧은 리뷰는 'Unknown'으로 처리
        if len(review) < 1:
            sentiment_results.append('Unknown')
        else:
            # 길이가 긴 리뷰는 처음 512 토큰만 사용
            result = classifier(review[:512])[0]
            sentiment_results.append(result['label'])
    except Exception as e:
        print(f"Error processing review: {e}")
        sentiment_results.append('Error')

# 분석 결과를 새로운 'senti_2' 열에 추가
df['senti_2'] = sentiment_results

# 결과 저장
df.to_csv("CrowledData_with_sentiment.csv", index=False, encoding='utf-8-sig')

# 결과 확인
print(df[['review', 'senti_2']].head())
print(f"분석 완료! 'senti_2' 열이 추가된 데이터가 저장되었습니다.")

Device set to use cpu
100%|██████████| 16045/16045 [19:50<00:00, 13.48it/s] 


                                              review   senti_2
0  자리는 불편한데 빵은 맛있는 카페\n\n롤토르타(레몬)\n레몬향이 한가득 나서 초반...  negative
1  아침먹고 투또톤토에 갔어요~ 여유롭게 시간 보내기가 좋아요~ 포장하면 할인되고요~ㅎ...  positive
2  매장이 엄청 넓고 인테리어도 모던하니 예뻤어요ㅎㅎ\n커피에 진짜 진심인 카페였습니다...  positive
3  건대입구 오픈했을때부터 애용하는 카페 투또톤토입니다!(아직도 이름이 헷갈려요)\n\...  positive
4  아침 일찍오니 빵 종류가 많네요. 좋아하는 말차빵이 있어 얼른 get했어요. 샐러드...  positive
분석 완료! 'senti_2' 열이 추가된 데이터가 저장되었습니다.


1열 제거 3열 제거 4열 제거 6열 제거 7열 제거

In [5]:
import pandas as pd

file_path = "CrowledData.csv"
data = pd.read_csv(file_path, encoding='utf-8-sig')

columns_to_remove = [0,2,3,5,6]
data = data.drop(data.columns[columns_to_remove], axis=1)

output_file = "R_Data_1.csv"
data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"지정된 열이 제거된 데이터가 저장되었습니다: {output_file}")

지정된 열이 제거된 데이터가 저장되었습니다: R_Data_1.csv


3열의 값이 "negative"인 행을 제거

In [3]:
import pandas as pd

file_path = "R_Data_1.csv"
data = pd.read_csv(file_path, encoding='utf-8-sig')

data = data[data.iloc[:, 2] != "negative"]

output_file = "R_Data_2.csv"
data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"3열의 값이 'negative'인 행이 제거된 데이터가 저장되었습니다: {output_file}")

3열의 값이 'negative'인 행이 제거된 데이터가 저장되었습니다: R_Data_2.csv


In [6]:
import pandas as pd

# 데이터 로드
file_path = "R_Data_2.csv"
data = pd.read_csv(file_path, encoding='utf-8-sig')

# 각 카페 이름별로 리뷰 병합
merged_reviews = data.groupby('카페이름')['review'].apply(lambda x: ' '.join(x)).reset_index()

# 결과 저장
output_file = "R_Data_3.csv"
merged_reviews.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"카페별 리뷰가 병합된 데이터가 저장되었습니다: {output_file}")

카페별 리뷰가 병합된 데이터가 저장되었습니다: R_Data_3.csv


행개수 찾기

In [ ]:
import pandas as pd
df = pd.read_csv("R_Data_3.csv", encoding='utf-8-sig')
print(f"데이터프레임의 총 행 수: {len(df)}")

데이터프레임의 총 행 수: 215
